In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
from tensorflow.keras.applications.densenet import DenseNet169
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_list_ds = tf.data.Dataset.list_files(str('../input/mura-dataset/train/*/*'))
valid_list_ds = tf.data.Dataset.list_files(str('../input/mura-dataset/valid/*/*'))

for f in train_list_ds.take(5):
  print(f.numpy())


In [ ]:
CLASS_NAMES = np.array(['Abnormal','Normal'])

def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    if parts[-2] == 'Abnormal':
        return [1,0]
    else:
        return [0,1]

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=0,dtype=tf.dtypes.uint8)
    img = tf.keras.backend.cast(img,dtype='float32')
    # resize the image to the desired size.
    return tf.image.resize(img,[448,448])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    # Normalizing input image with respect to the whole dataset
    img = img - 50.1428
    img = img/ 0.1812
    return img, label

In [ ]:
train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
valid_labeled_ds = valid_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

def prepare_for_training(ds, BATCH_SIZE = 50, shuffle_buffer_size=2000):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
train_ds = prepare_for_training(train_labeled_ds)
valid_ds = prepare_for_training(valid_labeled_ds)

# Creating a batch to visualize input data
image_batch, label_batch = next(iter(train_ds))

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(np.squeeze(image_batch[n]),cmap='gray')
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')

show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
pre_trained_model = DenseNet169(input_shape= (448,448,1), weights= None,include_top= False)

for layer in pre_trained_model.layers[:-70]:
    layer.trainable = False

In [ ]:
x = pre_trained_model.output
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.models.Model(pre_trained_model.input, x)
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

red_lr_plat = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                                   factor=0.5,
                                                   patience=2, 
                                                   mode='auto',
                                                   min_delta=0.001,
                                                   cooldown=0,
                                                   verbose=1,
                                                   min_lr=0)
callbacks_list = [red_lr_plat]

h = model.fit(train_ds,
          steps_per_epoch=720,
          validation_data= valid_ds,
          validation_steps=1,
          epochs=50,
          callbacks=callbacks_list)